# Setting DEM for Shetran

This notebook was created to demostrate the process of creating the DEM file use in SHETRAN based in the catchment extent

---

#### Author: 
                LF Velasquez - Newcastle University

#### Date:
                May 2023

#### Version:
                1.0

#### Notes:
                - To get jupyter env version type `!jupyter --version` in a python cell
            
#### Jupyter version:

#### Python version:

---

# Notebook set-up

## 1. Setting Python Modules

In [1]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import rasterstats
import rioxarray

# ODC modules
import datacube

# Custom modules
import utils

# Modules for config.ini
from configparser import ConfigParser
config = ConfigParser()

## 2. Global variables

In [2]:
# Setting the path to the work environment
dir_abs = Path().resolve().parent.parent
dir_abs

# Read config file values
config.read('config.ini')

# Setting CRS
crs_global = config.getint('crs_setting', 'GLB')
crs_local = config.getint('crs_setting', 'COL')

# Open Data Cube Product
dc_data = config.get('dc_product', 'DEM')

## 3. Start of Process

### Reading shapefile

In [3]:
# Read shp file to geopandas dataframe
grid_path = Path(dir_abs / 'shetran_data/active_data/final_mask_wgs84.shp')
grid = gpd.read_file(grid_path)

xmin, ymin, xmax, ymax = grid.total_bounds
grid

,intersect,SHETRAN_ID,geometry
0,0,-9999,"POLYGON ((-76.70463 3.59102, -76.68665 3.59107..."
1,0,-9999,"POLYGON ((-76.70458 3.57295, -76.68660 3.57300..."
2,0,-9999,"POLYGON ((-76.70453 3.55488, -76.68655 3.55493..."
3,0,-9999,"POLYGON ((-76.70448 3.53681, -76.68650 3.53687..."
4,0,-9999,"POLYGON ((-76.70443 3.51875, -76.68644 3.51880..."
...,...,...,...
793,1,0,"POLYGON ((-76.03853 3.30348, -76.02054 3.30351..."
794,0,-9999,"POLYGON ((-76.03849 3.28540, -76.02051 3.28544..."
795,0,-9999,"POLYGON ((-76.03846 3.26733, -76.02047 3.26736..."
796,0,-9999,"POLYGON ((-76.03842 3.24925, -76.02044 3.24929..."


### Reading DEM data from ODC

In [4]:
# Set ODC application
dc = datacube.Datacube(app="dem_cop")

# # Set dask client
# client = dask.distributed.Client()
# display(client)

# Get data
# Load data from the datacube
buffer = 0.125
ds = dc.load(product=dc_data,
             lat=(ymin - buffer, ymax + buffer),
             lon=(xmin - buffer, xmax + buffer),
            #  time=('2000-01-01T12:00:00'),
            #  dask_chunks={'time': 1, 'longitude': 200, 'latitude': 200}
             )

# Print output data
ds

<xarray.Dataset>
Dimensions:      (time: 1, latitude: 2273, longitude: 3364)
Coordinates:
  * time         (time) datetime64[ns] 2000-01-01T12:00:00
  * latitude     (latitude) float64 3.718 3.717 3.717 ... 3.087 3.087 3.087
  * longitude    (longitude) float64 -76.83 -76.83 -76.83 ... -75.9 -75.9 -75.9
    spatial_ref  int32 4326
Data variables:
    elevation    (time, latitude, longitude) float32 1.586e+03 ... 2.737e+03
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

### Create temp .tif file
This uses the information read from ODC

In [5]:
_tif = Path(dir_abs / 'shetran_data/temp_data/dem_cop.tif')
ds['elevation'].rio.to_raster(_tif)


### Perform zonal stats for the catcment

In [6]:
# Find the mean elevation for each grid in the catchment
# This produces a dict following the same order as the grid geodataframe
catchm_stats_mean = rasterstats.zonal_stats(str(grid_path), str(_tif), stats="mean", all_touched=True, nodata=-9999)
catchm_stats_min = rasterstats.zonal_stats(str(grid_path), str(_tif), stats="min", all_touched=True, nodata=-9999)

# Change the dictionary to a list to be added to the grid geodataframe
elevation_mean = [x['mean'] for x in catchm_stats_mean]
elevation_min = [x['min'] for x in catchm_stats_min]

# Add the elevation mean and min to grid geodataframe
grid['elevation_mean'] = elevation_mean
grid['elevation_min'] = elevation_min

grid

,intersect,SHETRAN_ID,geometry,elevation_mean,elevation_min
0,0,-9999,"POLYGON ((-76.70463 3.59102, -76.68665 3.59107...",1299.584022,1153.129517
1,0,-9999,"POLYGON ((-76.70458 3.57295, -76.68660 3.57300...",1319.633379,1184.785400
2,0,-9999,"POLYGON ((-76.70453 3.55488, -76.68655 3.55493...",1495.676997,1291.165405
3,0,-9999,"POLYGON ((-76.70448 3.53681, -76.68650 3.53687...",1638.791436,1427.226562
4,0,-9999,"POLYGON ((-76.70443 3.51875, -76.68644 3.51880...",1762.051632,1563.753052
...,...,...,...,...,...
793,1,0,"POLYGON ((-76.03853 3.30348, -76.02054 3.30351...",3677.720845,3311.380371
794,0,-9999,"POLYGON ((-76.03849 3.28540, -76.02051 3.28544...",3617.588154,3335.616699
795,0,-9999,"POLYGON ((-76.03846 3.26733, -76.02047 3.26736...",3901.984848,3720.500000
796,0,-9999,"POLYGON ((-76.03842 3.24925, -76.02044 3.24929...",3789.283517,3564.845947


### Delete temp .tif file
This avoid over-bloating in case someone forgets to delete the file

In [7]:
utils.file_remove(_tif)

dem_cop.tif DELETED!


### Working with the DEM geodataframe to set the final output

In [8]:
# Get the centroid for each grid
grid_bg = grid.to_crs(crs_local)
grid_bg['centroid'] = grid_bg['geometry'].centroid

# Create lat (Y) and lon (X) columns
grid_bg['lat'] = grid_bg['centroid'].y.astype(int)
grid_bg['lon'] = grid_bg['centroid'].x.astype(int)

# Change geopandas to pandas ready to create csv file for the mean and min elevation
df_grid_mean = pd.DataFrame(grid_bg[['elevation_mean', 'lat', 'lon']].copy())
df_grid_min = pd.DataFrame(grid_bg[['elevation_min', 'lat', 'lon']].copy())

# Pivoting dataframe to replicate SHETRAN format
# Pivoting dataframe using lon as column and lat as row
df_pivot_mean = df_grid_mean.pivot(index='lat', columns='lon', values='elevation_mean')
df_pivot_mean = df_pivot_mean.sort_index(ascending=False).round(0).astype(int)

df_pivot_min = df_grid_min.pivot(index='lat', columns='lon', values='elevation_min')
df_pivot_min = df_pivot_min.sort_index(ascending=False).round(0).astype(int)

df_pivot_mean

lon,709018,711018,713018,715018,717018,719018,721018,723018,725018,727018,...,765018,767018,769018,771018,773018,775018,777018,779018,781018,783018
lat,,,,,,,,,,,,,,,,,,,,,
888266,1300,1265,1287,1398,1568,1737,1757,2021,1763,1460,...,1311,1740,2271,1936,2043,2377,2321,2347,2734,3432
886266,1320,1356,1348,1404,1552,1779,1992,1852,1740,1469,...,1369,1718,2203,2017,2262,2612,2742,2926,2950,3391
884266,1496,1446,1431,1439,1615,1902,2030,1774,1425,1265,...,1365,1686,2185,2203,2496,2731,3088,3342,3611,3608
882266,1639,1604,1546,1510,1812,1865,2009,1840,1457,1141,...,1386,1662,2066,2124,2324,2842,3396,3570,3812,3575
880266,1762,1736,1652,1602,1899,1707,1911,1707,1566,1225,...,1569,1871,1983,2020,2344,2660,3056,3731,3920,3706
878266,1840,1924,1829,1812,2020,1654,1650,1526,1365,1118,...,1583,2068,2442,2204,2735,3189,3467,3567,3699,3851
876266,2005,2151,2053,1970,1851,1638,1412,1324,1265,1090,...,1568,2053,2573,2521,2895,3227,3746,3830,3730,3842
874266,2358,2233,2021,1864,1717,1471,1296,1161,1097,1020,...,1545,2082,2342,2239,2566,2969,3517,3838,3896,3830
872266,2781,2466,2261,1973,1750,1600,1392,1287,1096,977,...,1551,1968,2010,2053,2466,2716,2912,3464,3723,3735


### Saving the final outpt as csv file

In [9]:
# Creating the text file for min elevation
utils.shetran_csv_file(dir_abs, 'final_dem_min_SHETRAN', df_pivot_min, 'd')
print('SHETRAN min elevation file created')

utils.shetran_csv_file(dir_abs, 'final_dem_mean_SHETRAN', df_pivot_mean, 'd')
print('SHETRAN mean elevation file created')


SHETRAN min elevation file created
SHETRAN mean elevation file created
